In [5]:
from pydantic import BaseModel, ValiadteError
from datetime import datetime
from typing import List, Optional

class User(BaseModel):
    id: int # 必填字段
    name: str = "John"  # 有默认值，选填字段
    signup_ts: Optional[datetime] = None    
    friends: List[int] = [] # 列表中的元素需要是int类型或可以转换为int类型

extrnal_data = {
    "id": 123,
    "name": "John Doe",
    "signup_ts": "2020-01-01 00:00",
    "friends": [1, 2, '3']
}
user = User(**extrnal_data)
print(user.id, user.friends)
print(repr(user.singup_ts))
print(user.dict())


123 [1, 2, 3]
datetime.datetime(2020, 1, 1, 0, 0)
{'id': 123, 'name': 'John Doe', 'singup_ts': datetime.datetime(2020, 1, 1, 0, 0), 'friends': [1, 2, 3]}


In [9]:
from pydantic import ValidationError
try:
    User(id = 1, name="John", signup_ts=datetime.today, friends=[1, 2, 'a'])
except ValidationError as e:
    print(e.json())


[
  {
    "loc": [
      "friends",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]


In [14]:
print(user.dict())
print(user.json())
print(user.copy())  # 这里是浅复制
print(User.parse_obj(extrnal_data))
print(User.parse_raw('{"id": 123, "name": "John Doe", "singup_ts": "2020-01-01T00:00:00", "friends": [1, 2, 3]}'))

{'id': 123, 'name': 'John Doe', 'singup_ts': datetime.datetime(2020, 1, 1, 0, 0), 'friends': [1, 2, 3]}
{"id": 123, "name": "John Doe", "singup_ts": "2020-01-01T00:00:00", "friends": [1, 2, 3]}
id=123 name='John Doe' singup_ts=datetime.datetime(2020, 1, 1, 0, 0) friends=[1, 2, 3]
id=123 name='John Doe' singup_ts=datetime.datetime(2020, 1, 1, 0, 0) friends=[1, 2, 3]
id=123 name='John Doe' singup_ts=datetime.datetime(2020, 1, 1, 0, 0) friends=[1, 2, 3]


In [16]:
print(user.schema())
print(user.schema_json())
User.construct()    # 不校验数据，直接创建模型类

{'title': 'User', 'type': 'object', 'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'title': 'Name', 'default': 'John', 'type': 'string'}, 'singup_ts': {'title': 'Singup Ts', 'type': 'string', 'format': 'date-time'}, 'friends': {'title': 'Friends', 'default': [], 'type': 'array', 'items': {'type': 'integer'}}}, 'required': ['id']}
{"title": "User", "type": "object", "properties": {"id": {"title": "Id", "type": "integer"}, "name": {"title": "Name", "default": "John", "type": "string"}, "singup_ts": {"title": "Singup Ts", "type": "string", "format": "date-time"}, "friends": {"title": "Friends", "default": [], "type": "array", "items": {"type": "integer"}}}, "required": ["id"]}


In [18]:
print(User.__fields__.keys())   # 字段

dict_keys(['id', 'name', 'singup_ts', 'friends'])


# 递归模型

In [21]:
class Sound(BaseModel):
    sound: str


class Dog(BaseModel):
    birthday: date
    weight: float = Optional[None]
    sound: List[Sound]  # 不同的狗有不同的叫声。递归模型（Recursive Models）就是指一个嵌套一个

In [28]:
from datetime import date

class Sound(BaseModel):
    sound: str

class Dog(BaseModel):
    birthday: date
    weight: float = Optional[None]
    sound : List[Sound]     # 递归嵌套，不同的狗有不同的叫声

dog = Dog(birthday = date.today(), weight = 6.66, sound = [Sound(sound = 'woof'), Sound(sound = 'bark'), Sound(sound = 'grrr')])
print(dog.dict())

{'birthday': datetime.date(2021, 12, 22), 'sound': [{'sound': 'woof'}, {'sound': 'bark'}, {'sound': 'grrr'}]}


# ORM模型：从类实例创建符合ORM对象的模型

In [33]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import constr

Base = declarative_base()

class CompanyOrm(Base):
    __tablename__ = 'companies'
    id = Column(Integer, primary_key=True,nullable=False)
    public_key = Column(String(20), index = True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))

class CompanyMode(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: List[constr(max_length=255)]

    class Config:
        orm_mode = True

co_orm = CompanyOrm(
    id = 123,
    public_key = '12345678901230',
    name = 'Google',
    domains = ['google.com', 'google.co.jp']
)

print(CompanyMode.from_orm(co_orm))

id=123 public_key='12345678901230' name='Google' domains=['google.com', 'google.co.jp']


# Pydantic支撑的字段类型

# 官方文档：https://pydantic-docs.helpmanual.io/usage/types/